The following code will require the user to ctrl-a the webpage they view the list view of the course schedule on QUEST UWATERLOO and copy-paste the text onto a text file. After reading the text file, the program will output an ics file which would allow the user to upload the course dates+times onto a calendar app like Outlook.

In [ ]:
import pandas as pd
import numpy as np
import pytz
from icalendar import Calendar, Event
import uuid
from dateutil.rrule import rrule, WEEKLY
from icalendar.prop import vRecur
import datetime

# Initialize an empty list to store the lines of data from the file
data = []

# Open the file containing the course data and read each line
with open('texttoread.txt', 'r') as file:
    # Read each line in the file
    for line in file:
        if line.strip():  # Only add non-empty lines
            data.append(line)

# Convert the list of lines into a pandas DataFrame for easier manipulation
data = pd.DataFrame(data, columns=[0], dtype='string')

# Find the indices where chunks of data start and end based on 'Status' and 'Printer Friendly Page'
begin_chunk = data.index[data.iloc[:, 0].str.contains('Status')].to_numpy() - 1
end_chunk = begin_chunk[1:]  # Set end indices for the chunks
last_elem = data.index[data.iloc[:, 0].str.contains('Printer Friendly Page')].to_numpy()
end_chunk = np.append(end_chunk, last_elem[0])  # Include the last element for the final chunk
chunk_indices = np.vstack((begin_chunk, end_chunk)).T  # Create pairs of start and end indices for each chunk

# Initialize a dictionary to hold course data organized by chunk name
list_courses = {}

# Iterate through each chunk and process it
for chunk in chunk_indices:
    chunk_table = data.iloc[chunk[0]:chunk[1], :]  # Slice the DataFrame to get the chunk
    chunk_table_name = chunk_table.iloc[0].to_string(index=False)  # Use the first row as the chunk name
    chunk_table = chunk_table.drop(chunk_table.index[0])  # Drop the first row (which contains the name)
    chunk_table = chunk_table.reset_index(drop=True)  # Reset index after dropping rows
    list_courses[chunk_table_name] = chunk_table  # Store the chunk in the dictionary

# Process each course in the list_courses dictionary
for key, course in list_courses.items():
    # Find the index of the row containing the table header
    id = course.index[course.iloc[:, 0].str.contains('Class Nbr')].to_list()
    
    # Drop rows before the table header and process columns
    course = course.drop(index=list(range(0, id[0])))
    course = course.iloc[:, 0].str.split(pat=r'\s*\t\s*', expand=True)  # Split the line into columns for each course detail
    course.columns = course.iloc[0, :]  # Set the first row as the column headers
    course = course.drop(index=id[0])  # Drop the header row
    course = course.reset_index(drop=True)  # Reset index

    # Identify rows with date range (Start/End Date) and split into separate columns
    id1 = course.index[course.iloc[:, 0].str.contains(r'\d{2}/\d{2}/\d{4} - \d{2}/\d{2}/\d{4}')].to_list()
    id1_len = len(id1)

    # Fill in the date range across rows for each course component (e.g., Section)
    for i in range(6, -1, -1):  # Iterate backwards to propagate dates
        course.iloc[0:id1_len, i] = course.iloc[id1, 0]
        for j in id1:
            if j != 0:
                course.iloc[j, 0] = np.nan  # Set previous rows to NaN to avoid duplicate values
        id1 = [x - 1 if x - 1 >= 0 else x for x in id1]  # Adjust the indices

    # Drop rows that are completely empty and forward-fill missing values
    course = course.dropna(how='all')
    course = course.ffill()

    # Update the list of courses with the cleaned course data
    list_courses[key] = course
# Concatenate all course chunks into one DataFrame and reset index
df = pd.concat(list_courses)
df = df.reset_index()
df = df.drop(df.columns[1], axis=1)  # Drop the redundant index column
df = df.rename(columns={df.columns[0]: "Course"})  # Rename first column to 'Course'
df.columns = df.columns.str.replace('\n', '')  # Remove any newline characters in column names
df = df.map(lambda x: str(x).replace('\n', ''))  # Clean up the data by removing newlines

# Case for ONLINE classes
df.loc[df['Room'].str.contains('ONLN'),'Days & Times'] = 'MTWThF 12:00AM - 11:59PM'

# Split 'Start/End Date' column into 'Start Date' and 'End Date'
df[['Start Date', 'End Date']] = df['Start/End Date'].str.split(pat=r'\s*-\s*', expand=True)
df = df.drop('Start/End Date', axis=1)

# Split 'Days & Times' column into separate columns for 'Days', 'Start Time', and 'End Time'
df['Days & Times'] = df['Days & Times'].str.replace(r'\s+|-', '', regex=True)

days_times = df['Days & Times'].str.extract(pat=r'([A-Za-z]+)\s*(\d{1,2}:\d{2}(?:[APMapm]{2})?)\s*(\d{1,2}:\d{2}(?:[APMapm]{2})?)')
days_times = days_times.rename(columns={days_times.columns[0]: 'Days', days_times.columns[1]: 'Start Time', days_times.columns[2]: 'End Time'})
df = pd.concat([df, days_times], axis=1)
df = df.drop('Days & Times', axis=1)  # Drop the original 'Days & Times' column
# Extract days from 'Days' column and convert to list of weekday numbers
df['Days'] = df['Days'].str.findall(r'[A-Z][^A-Z]*')
# Split 'Course' column into 'Course Code' and 'Course Name'
df[['Course Code', 'Course Name']] = df['Course'].str.extract(r'^(.+?)\s*-\s*(.+)$')
df = df.drop('Course', axis=1)  # Drop the original 'Course' column
# Convert columns to appropriate data types (e.g., integers, strings)
df = df.astype({'Course Code': 'string', 'Course Name': 'string', 'Class Nbr': 'int32', 'Section': 'string', 'Component': 'string', 'Room': 'string', 'Instructor': 'string'})
df['Start Date'] = pd.to_datetime(df['Start Date'].str.strip(), format='mixed')  # Convert 'Start Date' to datetime
df['Start Time'] = pd.to_datetime(df['Start Time'].str.strip(), format='mixed')  # Convert 'Start Time' to datetime
df['Start DateTime'] = df['Start Date'] + pd.to_timedelta(df['Start Time'].dt.strftime('%H:%M:%S'))  # Combine date and time into datetime object

df['End Date'] = pd.to_datetime(df['End Date'].str.strip(), format='mixed')  # Convert 'End Date' to datetime
df['End Time'] = pd.to_datetime(df['End Time'].str.strip(), format='mixed')  # Convert 'End Time' to datetime
df['End DateTime'] = df['End Date'] + pd.to_timedelta(df['End Time'].dt.strftime('%H:%M:%S'))  # Combine date and time into datetime object


# Map days of the week (e.g., 'M' -> 0, 'T' -> 1, etc.)
day_map = {'M': 0, 'T': 1, 'W': 2, 'Th': 3, 'F': 4, 'S': 5, 'Su': 6}
df = df.assign(Days=[[day_map[k] for k in row] for row in df.Days])  # Assign the numerical representation of days

if df['Start Date'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['Start Date'] = df['Start Date'].dt.tz_localize('America/Toronto')
if df['Start Time'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['Start Time'] = df['Start Time'].dt.tz_localize('America/Toronto')
if df['Start DateTime'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['Start DateTime'] = df['Start DateTime'].dt.tz_localize('America/Toronto')
if df['End Date'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['End Date'] = df['End Date'].dt.tz_localize('America/Toronto')
if df['End Time'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['End Time'] = df['End Time'].dt.tz_localize('America/Toronto')
if df['End DateTime'].dt.tz is None:
    # Now convert it to Toronto timezone
    df['End DateTime'] = df['End DateTime'].dt.tz_localize('America/Toronto')

# Create an iCalendar (ICS) calendar object
cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.dk//')
cal.add('version', '2.0')

# Loop through each course and create an event for it
for index, course in df.iterrows():
    event = Event()

    event.add('summary', course['Course Code'] + ' ' + course['Component'])  # Add course summary (e.g., course code and component)
    event.add('location', course['Room'])  # Add course room
    event.add('organizer', course['Instructor'])  # Add course instructor
    event.add('categories', course['Component'])  # Add course component as category
    event.add('description', course['Course Name'])  # Add course name as description

    # Add start and end times
    if abs(course['End Time']-course['Start Time'])>datetime.timedelta(hours=8): #checks if the event is all day which would likely be a course longer than 8 hours
        event.add('dtstart', course['Start DateTime'])
        event.add('dtend', course['Start Date'] + pd.to_timedelta(pd.Timedelta(days=1)))
    else:
        event.add('dtstart', course['Start DateTime'])
        event.add('dtend', course['Start Date'] + pd.to_timedelta(course['End Time'].strftime('%H:%M:%S')))
    event.add('dtstamp', course['Start Date'])  # Add timestamp for the event
    event.add('uid', str(uuid.uuid4()))  # Generate a unique ID for the event
    
    # Set up recurrence rule (weekly occurrence of the class until the end date)
    rrule_string = str(rrule(freq=WEEKLY, dtstart=course['Start DateTime'], until=course['End DateTime'], byweekday=course['Days'])).split("\n")[1].replace('RRULE:', '')
    rep_rule = vRecur.from_ical(rrule_string)  # Convert recurrence rule string into vRecur object
    event.add("RRULE", rep_rule)  # Add recurrence rule to the event

    cal.add_component(event)  # Add the event to the calendar

cal.add_missing_timezones()

# Write the calendar data to an ICS file
with open('schedule.ics', 'wb') as f:
    f.write(cal.to_ical())  # Write the iCalendar data to the file
    f.close()  # Close the file